In [1]:
import numpy as np
import torch

In [88]:
class Perceptron:
    """
    A simple Perceptron implementation.

    The Perceptron is a fundamental binary classifier that uses a linear decision boundary 
    to classify data points. This implementation includes basic functionality of a Perceptron 
    with customizable activation functions.

    Parameters
    ----------
    dimension_data : int
        The number of features (excluding the bias) in the input data.

    Attributes
    ----------
    activation_function : callable
        The function used for activation.
    diff_activation_function : callable
        The derivative of the activation function.
    bias : float
        The bias term added to the input data.
    weights : np.ndarray
        The weights of the perceptron, initialized to None.
    activation_function_name : str
        The name of the activation function currently in use.

    Methods
    -------
    set_activation_function(name='sigmoid')
        Set the activation function by name.
    insert_bias(x)
        Insert a bias term into the input vector.
    loss_function(weights, x, y)
        Calculate the loss function given weights, inputs, and the target.
    compute_loss(predictions, targets)
        Compute the loss between predictions and targets.
    forward(X, y, epochs=100, learning_rate=0.01)
        Run the forward training loop for the perceptron.
    """
    def __init__(self, dimension_data) -> None:
        self.activation_function = None
        self.diff_activation_function = None
        self.bias = 1
        self.weights = None
        self.activation_function_name = 'sigmoid'
    
    def set_activation_function(self, name = 'sigmoid'):
        """
        Set the activation function for the perceptron.

        Parameters
        ----------
        name : str, optional
            The name of the activation function. Supported values are 'relu', 'sigmoid', and 'tanh'.
            Default is 'sigmoid'.

        Raises
        ------
        ValueError
            If the specified activation function name is not supported.
        """
        self.activation_function_name = name.lower()
        if name.lower() == 'relu':
            self.activation_function = lambda x: np.maximum(0, x)
            self.diff_activation_function = lambda x: np.where(x > 0, 1, 0)
        elif name.lower() == 'sigmoid':
            self.activation_function = lambda x: 1 / (1 + np.exp(-x))
            self.diff_activation_function = lambda x: self.activation_function(x) * (1 - self.activation_function(x))
        elif name.lower() == 'tanh':
            self.activation_function = lambda x: np.tanh(x)
            self.diff_activation_function = lambda x: 1 - np.tanh(x)**2
        else:
            raise ValueError("Unsupported activation function. Choose 'relu', 'sigmoid', or 'tanh'.")
        
    def insert_bias(self, x):
        return np.insert(x, 0, self.bias)  # Always insert bias at index 0
    
    def loss_function(self, weights, x, y):
        prediction = self.activation_function(np.dot(weights, x))
        return prediction - y

    def compute_loss(self, predictions, targets):
        # Check which activation function is used based on a stored name attribute
        if self.activation_function_name == 'sigmoid':
            # Binary cross-entropy for sigmoid activation
            return -np.mean(targets * np.log(predictions + 1e-9) + (1 - targets) * np.log(1 - predictions + 1e-9))
        elif self.activation_function_name == 'relu' or self.activation_function_name == 'tanh':
            # Mean squared error for ReLU or tanh activations in a regression context
            return np.mean((predictions - targets) ** 2)
        else:
            raise ValueError(f"Unsupported or undefined activation function name: {self.activation_function_name}")

    
                
    def forward(self, X, y, epochs=100, learning_rate=0.01):
        """
        Perform the forward pass and update weights based on the training data.
    
        Parameters
        ----------
        X : np.ndarray
            The input data matrix.
        y : np.ndarray
            The target output vector.
        epochs : int, optional
            The number of epochs to train the model. Default is 100.
        learning_rate : float, optional
            The learning rate for weight updates. Default is 0.01.
    
        Notes
        -----
        This method updates the weights based on the loss gradient and prints the average loss per epoch.
        """
        # Start random weights
        self.weights = np.random.randn(X.shape[1] + 1)
        # Ensure X has bias terms inserted; reshape X to include bias as the first column
        X = np.hstack((np.ones((X.shape[0], 1)), X))  # Add a column of ones for the bias
        
        
        for epoch in range(epochs):
            total_loss = 0
            for i in range(len(X)):
                z = np.dot(X[i], self.weights)  # Calculate the linear combination for each instance
                predictions = self.activation_function(z)
                loss = self.compute_loss(np.array([predictions]), np.array([y[i]]))  # Compute loss for the current instance
                total_loss += loss
                error = predictions - y[i]  # Calculate error for gradient
                gradient = error * self.diff_activation_function(z) * X[i]
                self.weights -= learning_rate * gradient  # Properly adjust sign for weight update
    
            average_loss = total_loss / len(X)
            print(f'Epoch {epoch + 1}, Average Loss: {average_loss}')
    




In [89]:
# Generate random data points and random binary targets
np.random.seed(0)  # For reproducibility
X = np.random.randn(100, 1)  # 100 unidimensional data points
y = np.random.randint(0, 2, size=(100,))  # Binary targets (0 or 1)

In [90]:
X.shape

(100, 1)

In [91]:
perceptron = Perceptron(dimension_data=1)  # Initialize with one feature
perceptron.set_activation_function('sigmoid')  # Set sigmoid for binary classification

In [92]:
perceptron.forward(X, y, epochs=100, learning_rate=0.01)


Epoch 1, Average Loss: 0.8228163422351933
Epoch 2, Average Loss: 0.8128472872298872
Epoch 3, Average Loss: 0.8033136807224527
Epoch 4, Average Loss: 0.7942128136727276
Epoch 5, Average Loss: 0.7855423328295688
Epoch 6, Average Loss: 0.7772999248762433
Epoch 7, Average Loss: 0.7694830028888946
Epoch 8, Average Loss: 0.7620884058458808
Epoch 9, Average Loss: 0.755112122284088
Epoch 10, Average Loss: 0.7485490490350903
Epoch 11, Average Loss: 0.74239279511012
Epoch 12, Average Loss: 0.7366355391494587
Epoch 13, Average Loss: 0.7312679464319715
Epoch 14, Average Loss: 0.7262791483952885
Epoch 15, Average Loss: 0.7216567841937461
Epoch 16, Average Loss: 0.7173871003396975
Epoch 17, Average Loss: 0.7134551012799834
Epoch 18, Average Loss: 0.7098447411696731
Epoch 19, Average Loss: 0.706539145357897
Epoch 20, Average Loss: 0.7035208493211671
Epoch 21, Average Loss: 0.7007720429685526
Epoch 22, Average Loss: 0.6982748092864469
Epoch 23, Average Loss: 0.6960113479892187
Epoch 24, Average Loss: 